In [1]:
import sys
import os

# Get the path to src/ relative to this notebook
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, "../../src"))

# Add to Python path if not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from utils.chelsa import extract_param
from datasource.chelsa import s3, layer, sampling

# Generate CHELSA cube

In [ ]:
param_file = "prototype_param.yaml"
chelsa_month_param = extract_param.read_chelsa_month_param(param_file, param_path = "../../config")
chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = "../../config")
chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = "../../config")
chelsa_clim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = "../../config")
chelsa_clim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = "../../config")

In [ ]:
def da_layer_constructor(data_layer_func, param):
    static_param = list(param.values())[1:]
    data_arrays = []
    for var in param["var"]:
        data_arrays.append(data_layer_func(var, *static_param))
    return data_arrays

In [ ]:
test = da_layer_constructor(layer.chelsa_month_ts, chelsa_month_param)

In [ ]:
test

In [ ]:
import xarray

def regrid_interp(
    da: xr.DataArray,
    new_lats: xr.DataArray,
    new_lons: xr.DataArray,
    method: str = "linear"
) -> xr.DataArray:
    """
    Upscale or downscale by interpolating `da` onto (new_lats, new_lons).
    `method` can be "linear", "nearest", or "cubic".
    """
    # Ensure monotonic coords
    da = da.sortby(["lat", "lon"])
    return da.interp(lat=new_lats, lon=new_lons, method=method)

def reproject_array(data_array, upscaled_long, upscaled_lat, method="cubic"):
    

In [ ]:
import numpy as np

def compare_coord_ticks(coord_ticks):
    ref = coord_ticks[0]
    differences = []  
    for i, arr in enumerate(coord_ticks):
        if len(arr) != len(ref):
            differences.append((i, "different length"))
        elif not np.array_equal(arr, ref):
            differences.append((i, "same length, different values")) 
    return differences

In [ ]:
long_coordinates = [da[1].long.values for da in test]
lat_coordinates = [da[1].lat.values for da in test]

def group_by_size(coordinates):
    #Retrieve size of coordinates
    coordinates_sizes = [len(coordinate_array) for coordinate_array in coordinates]
    #Group coordinates that are similar in size together
    co_idx = [[idx for idx, el in enumerate(coordinates_sizes) if el==size] for size in list(set(coordinates_sizes))]
    return co_idx

def check_homogeneity(coordinates):
    if len(coordinates)==1:
        return True
    else:
        
group_by_size(lat_coordinates)

In [ ]:
def harmonize_data_arrays(data_arrays, 
                          harmonization="upscale", method="linear", fill_na=True):
    if harmonization=="upscale":
        long_coordinates = [da.long.values for da in data_arrays]
        latitude_coordinates = [da.lat.values for da in data_arrays]
        target_long = 
        target_lat

In [ ]:
class spatiotemporal_cube():
    def __init__(self):
        pass

    def da_layer_constructor(self, data_layer_func, param):
        static_param = list(param.values())[1:]
        data_arrays = []
        for var in param["var"]:
            data_arrays.append(data_layer_func(var, *static_param))
        return data_arrays

    def da_concat(self, data_arrays, dim_name, coordinates):
        combined_data_array = xr.concat(data_arrays, dim=dim_name)
        combined_data_array = combined_data_array.assign_coords({dim_name: coordinates})
        return combined_data_array
        
    def regrid_spatial_coordinates(self, data_array, target_lat, target_long,
                                   harmonization="upscale", method="linear", fill_nearest=True):
        if harmonization=="upscale":
            upscaled_data_array = data_array.interp(lat=target_lat, long=target_long, method=method)
            if fill_nearest:
                upscaled_filled_data_array = upscaled_data_array.ffill(dim="lat").bfill(dim="lat").ffill(dim="long").bfill(dim="long")
                return upscaled_filled_data_array
            else:
                return upscaled_data_array

In [ ]:
chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = "../../config")
chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = "../../config")
chelsa_clim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = "../../config")
chelsa_clim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = "../../config")

In [116]:
import xarray as xr
import yaml

class spatiotemporal_cube():
    def __init__(self):
        pass

    def da_layer_constructor(self, data_layer_func, param):
        static_param = list(param.values())[1:]
        data_arrays = []
        for var in param["var"]:
            data_arrays.append(data_layer_func(var, *static_param))
        return data_arrays

    def da_concat(self, data_arrays, dim_name, coordinates):
        combined_data_array = xr.concat(data_arrays, dim=dim_name)
        combined_data_array = combined_data_array.assign_coords({dim_name: coordinates})
        return combined_data_array
        
    def regrid_spatial_coordinates(self, data_array, target_lat, target_long,
                                   harmonization="upscale", method="linear", fill_nearest=True):
        if harmonization=="upscale":
            upscaled_data_array = data_array.interp(lat=target_lat, long=target_long, method=method)
            if fill_nearest:
                upscaled_filled_data_array = upscaled_data_array.ffill(dim="lat").bfill(dim="lat").ffill(dim="long").bfill(dim="long")
                return upscaled_filled_data_array
            else:
                return upscaled_data_array
                
class chelsa_cube(spatiotemporal_cube):
    def __init__(self):
        self.layers = ["chelsa_month", 
                       "chelsa_clim_ref_period",
                       "chelsa_clim_ref_month",
                       "chelsa_clim_sim_period",
                       "chelsa_clim_sim_month"]
        self.data = []

    #generate monthly data for a set of variables
    def generate_chelsa_month_layer(self, param_file, param_path = "../../config"):
        chelsa_month_param = extract_param.read_chelsa_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_month_ts, chelsa_month_param)) #returns list of tuples (var_name, data)
        #Current assumption is that all variables will be requested
        #In future general harmonization functions will be implemented 
        #Select the coordinates from the second variable, clt is the only one that differs
        target_lat = data[1].lat.values
        target_long = data[1].long.values
        upscaled_clt = self.regrid_spatial_coordinates(data[0], target_lat, target_long)
        data = [upscaled_clt]+list(data[1:])
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_ref_period_layer(self, param_file, param_path="../../config"):
        chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_ref_period, chelsa_ref_period_param))
        upscale_idx = [var_names.index(el) for el in ['clt_max', 'clt_mean', 'clt_min', 'clt_range'] if el in var_names]
        data = list(data)
        if upscale_idx:
            #Select the idx of data array that does not need to be upscaled
            #The difference between sets generates the set containing idx that are not in upscale idx
            target_idx = list(set(list(range(len(var_names))))-set(upscale_idx))[0]
            target_lat = data[target_idx].lat.values
            target_long = data[target_idx].long.values
            upscaled_da = [self.regrid_spatial_coordinates(data[idx], target_lat, target_long) for idx in upscale_idx]
            for idx, da in zip(upscale_idx, upscaled_da):
                data[idx] = da
        #return var_names, data
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_ref_month_layer(self, param_file, param_path="../../config"):
        chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_ref_month, chelsa_ref_month_param))
        #Current assumption is that all variables will be requested
        #In future general harmonization functions will be implemented 
        #Select the coordinates from the second variable, clt is the only one that differs
        target_lat = data[1].lat.values
        target_long = data[1].long.values
        upscaled_clt = self.regrid_spatial_coordinates(data[0], target_lat, target_long)
        data = [upscaled_clt]+list(data[1:])
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_sim_period_layer(self, param_file, param_path="../../config"):
        chelsa_sim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_sim_period, chelsa_sim_period_param))
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_sim_month_layer(self, param_file, param_path="../../config"):
        chelsa_sim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_sim_month, chelsa_sim_month_param))
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_cube(self, param_file, param_path="../../config"):
        param_filepath = os.path.join(param_path, param_file)
        with open(param_filepath) as f:
            param_dict = yaml.safe_load(f)
        enabled_layers = [param_dict["layers"][layer] for layer in self.layers]
        functions = [self.generate_chelsa_month_layer, 
                     self.generate_chelsa_ref_period_layer,
                     self.generate_chelsa_ref_month_layer,
                     self.generate_chelsa_sim_period_layer,
                     self.generate_chelsa_sim_month_layer]
        enabled_layers_names = [name for name, call in zip(self.layers, enabled_layers) if call]
        data = [func(param_file, param_path) for func, call in zip(functions, enabled_layers) if call]
        return dict(zip(enabled_layers_names, data))

In [108]:
test_chelsa_cube = chelsa_cube()

In [109]:
test_ds = test_chelsa_cube.generate_chelsa_cube("prototype_param.yaml")

-----Retrieving monthly CHELSA data for variable 'clt'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.16item/s]


-----Retrieving monthly CHELSA data for variable 'cmi'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.13item/s]


-----Retrieving monthly CHELSA data for variable 'hurs'-----


...

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.07item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmax'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.24item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmin'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:16<00:00,  5.37item/s]


In [110]:
test_ds

<xarray.Dataset> Size: 127MB
Dimensions:                 (time: 12, lat: 23, long: 69, variable: 87,
                             months: 12, year_range: 3, model_name: 5,
                             ensemble_member: 3, month: 2)
Coordinates:
  * time                    (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                     (lat) float64 184B 50.87 50.87 50.86 ... 50.7 50.69
  * long                    (long) float64 552B 4.176 4.184 ... 4.734 4.742
  * variable                (variable) <U16 6kB 'ai' 'bio1' ... 'vpd_range'
  * months                  (months) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * year_range              (year_range) <U9 108B '2011-2040' ... '2071-2100'
  * model_name              (model_name) <U13 260B 'gfdl-esm4' ... 'ukesm1-0-ll'
  * ensemble_member         (ensemble_member) <U6 72B 'ssp126' 'ssp370' 'ssp585'
  * month                   (month) int64 16B 1 2
Data variables:
    chelsa_month            (variable, time, lat, long) float64 13MB nan ... nan
    chelsa_clim_ref_period  (variable, lat, long) float64 1MB 1.18 ... 5.568e+03
    chelsa_clim_ref_month   (variable, months, lat, long) float64 13MB nan .....
    chelsa_clim_sim_period  (variable, year_range, model_name, ensemble_member, lat, long) float64 50MB ...
    chelsa_clim_sim_month   (variable, year_range, month, model_name, ensemble_member, lat, long) float32 50MB ...

In [111]:
test_layer_m = test_chelsa_cube.generate_chelsa_month_layer("prototype_param.yaml")

-----Retrieving monthly CHELSA data for variable 'clt'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  8.27item/s]


-----Retrieving monthly CHELSA data for variable 'cmi'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.32item/s]


-----Retrieving monthly CHELSA data for variable 'hurs'-----


...

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.13item/s]


-----Retrieving monthly CHELSA data for variable 'tasmin'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  8.11item/s]


-----Retrieving monthly CHELSA data for variable 'vpd'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.24item/s]


In [70]:
test_layer_ref_p = test_chelsa_cube.generate_chelsa_ref_period_layer("prototype_param.yaml")

-----Retrieving Reference Climatology CHELSA data for variable 'ai' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio10' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio11' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio12' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio13' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio14' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio15' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio16' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio17' in reference

In [20]:
test_layer_ref_m = test_chelsa_cube.generate_chelsa_ref_month_layer("prototype_param.yaml")

-----Retrieving monthly Reference Climatology CHELSA data for variable 'clt' in reference period 1981-2010-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.06item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'cmi' in reference period 1981-2010-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.34item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'hurs' in reference period 1981-2010-----


...

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.16item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'tasmin' in reference period 1981-2010-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.98item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'vpd' in reference period 1981-2010-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.87item/s]


In [26]:
test_layer_sim_p = test_chelsa_cube.generate_chelsa_sim_period_layer("prototype_param.yaml")

-----Retrieving Simulation (period) CHELSA data for variable 'bio10'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:05<00:00,  7.52item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'bio11'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:05<00:00,  7.56item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'bio12'-----


...

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:06<00:00,  6.63item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'scd'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:05<00:00,  7.57item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'swe'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:06<00:00,  6.87item/s]


In [27]:
test_sim_m = test_chelsa_cube.generate_chelsa_sim_month_layer("prototype_param.yaml")

-----Retrieving Simulation (monthly) CHELSA data for variable 'pr'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.05item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tas'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.27item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmax'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.24item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmin'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:16<00:00,  5.41item/s]


In [112]:
test_ds = xr.Dataset({"chelsa_month":test_layer_m, "chelsa_ref_period":test_layer_ref_p})

In [113]:
test_ds

<xarray.Dataset> Size: 14MB
Dimensions:            (time: 12, lat: 23, long: 69, variable: 87)
Coordinates:
  * time               (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long               (long) float64 552B 4.176 4.184 4.192 ... 4.734 4.742
  * variable           (variable) <U16 6kB 'ai' 'bio1' ... 'vpd_min' 'vpd_range'
Data variables:
    chelsa_month       (variable, time, lat, long) float64 13MB nan nan ... nan
    chelsa_ref_period  (variable, lat, long) float64 1MB 1.18 ... 5.568e+03

In [114]:
test_ds = xr.Dataset({})
test_ds["chelsa_month"]=test_layer_m
test_ds["chelsa_ref_period"]=test_layer_ref_p

In [115]:
test_ds

<xarray.Dataset> Size: 2MB
Dimensions:            (time: 12, lat: 23, long: 69, variable: 11)
Coordinates:
  * time               (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long               (long) float64 552B 4.176 4.184 4.192 ... 4.734 4.742
  * variable           (variable) <U7 308B 'clt' 'cmi' 'hurs' ... 'tasmin' 'vpd'
Data variables:
    chelsa_month       (variable, time, lat, long) float64 2MB 5.152e+03 ... ...
    chelsa_ref_period  (variable, lat, long) float64 140kB nan nan ... nan nan